# 🔁 Day 12: Transfer Learning with Pretrained CNNs

In this notebook, we'll use **Transfer Learning** with a **pretrained ResNet50** model to classify images from a subset of CIFAR-10.

### Goals:
- Load and preprocess CIFAR-10 dataset
- Use ResNet50 with pretrained ImageNet weights
- Train only the top layers on CIFAR-10
- Evaluate and visualize the results

In [ ]:
# 📦 Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
# 📥 Load and Preprocess CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Resize images to 224x224 as required by ResNet
x_train_resized = tf.image.resize(x_train, [224, 224])
x_test_resized = tf.image.resize(x_test, [224, 224])

x_train_resized = preprocess_input(x_train_resized)
x_test_resized = preprocess_input(x_test_resized)

y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

In [ ]:
# 🧠 Load Pretrained ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base layers

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# 🏋️ Train Model
history = model.fit(x_train_resized, y_train_cat,
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1)

In [ ]:
# 📈 Evaluate Model
test_loss, test_acc = model.evaluate(x_test_resized, y_test_cat)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
# 📊 Plot Training History
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()